In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
from scipy import stats

import tensorflow as tf
keras = tf.keras


ModuleNotFoundError: No module named 'tensorflow'